In [ ]:
# ==============================================================================
# SCRIPT DOWNSCALING & INTERPOLASI IDW (METRIC 1000 METER)
# Konteks: Skripsi Jariyan Arifudin - Jambi (UTM Zone 48S)
# ==============================================================================

import os
import glob
import numpy as np
import rasterio
from rasterio.transform import from_origin
from rasterio.mask import mask, geometry_mask # Added geometry_mask here
import geopandas as gpd
from scipy.spatial import cKDTree
from pyproj import Transformer # Library baru untuk konversi koordinat
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# --- 1. KONFIGURASI DIREKTORI & PARAMETER ---

INPUT_DIR = '/content/drive/My Drive/Colab Notebooks/Skripsi/CHIRPS v3/bias_corrected_ga/raster_2009_2020/'
ADMIN_PATH = '/content/drive/My Drive/Colab Notebooks/Skripsi/Batas Adm Jambi/Adm_Jambi_Prov.shp'

OUTPUT_DIR = os.path.join(INPUT_DIR, 'downscaled_1km_metric')
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- PARAMETER METRIK ---
TARGET_CRS = 'EPSG:32748' # UTM Zone 48S
TARGET_RES = 1000         # 1000 Meter
BUFFER_METER = 5000       # Buffer 5km di sekitar batas admin
IDW_POWER = 2
K_NEIGHBORS = 12
NODATA_VAL = -9999

In [ ]:
# --- 2. PERSIAPAN DATA SPASIAL (DILAKUKAN SEKALI DI AWAL) ---

print("=== MEMULAI PROSES DOWNSCALING (TARGET: 1000 METER / UTM 48S) ===")

# A. Siapkan Batas Wilayah (Reprojection ke UTM)
print(f"[1/4] Membaca Shapefile & Reproyeksi ke {TARGET_CRS}...")
try:
    jambi_gdf = gpd.read_file(ADMIN_PATH)
    jambi_gdf = jambi_gdf.to_crs(TARGET_CRS)
except Exception as e:
    print(f"[ERROR] Masalah pada shapefile: {e}")
    raise

=== MEMULAI PROSES DOWNSCALING (TARGET: 1000 METER / UTM 48S) ===
[1/4] Membaca Shapefile & Reproyeksi ke EPSG:32748...


In [ ]:
# B. Membuat Grid Target (Template Kosong)
print(f"[2/4] Membuat Grid Target Resolusi {TARGET_RES} Meter...")
minx, miny, maxx, maxy = jambi_gdf.total_bounds

# Membuat koordinat grid (Left, Top, Right, Bottom)
# Menggunakan kelipatan resolusi agar snap-to-grid rapi
grid_minx = np.floor((minx - BUFFER_METER) / TARGET_RES) * TARGET_RES
grid_maxy = np.ceil((maxy + BUFFER_METER) / TARGET_RES) * TARGET_RES

# Hitung dimensi output
width_px = int((maxx + BUFFER_METER - grid_minx) / TARGET_RES)
height_px = int((grid_maxy - (miny - BUFFER_METER)) / TARGET_RES)

# Membuat Transformasi Affine untuk Output (West, North, pixel_w, pixel_h)
# pixel_h negatif karena koordinat peta biasanya utara di atas (y menurun saat baris bertambah)
out_transform = from_origin(grid_minx, grid_maxy, TARGET_RES, TARGET_RES)

# Generate koordinat X dan Y untuk setiap piksel target
# Menggunakan logika affine transform untuk efisiensi memori
cols, rows = np.meshgrid(np.arange(width_px), np.arange(height_px))
xs_target, ys_target = rasterio.transform.xy(out_transform, rows, cols, offset='center')

# Flatten target coordinates untuk input cKDTree
flat_target_x = np.array(xs_target).flatten()
flat_target_y = np.array(ys_target).flatten()
target_coords = np.c_[flat_target_x, flat_target_y]

print(f"      Dimensi Grid: {height_px} Baris x {width_px} Kolom")

[2/4] Membuat Grid Target Resolusi 1000 Meter...
      Dimensi Grid: 236 Baris x 385 Kolom


In [ ]:
# C. Membuat Mask Geometri (Untuk Clipping in-memory)
print("      Membuat Masking Geometri di Memori...")
# geometry_mask menghasilkan True di LUAR shape, False di DALAM shape (invert=True membalik ini)
# Namun, rasterio geometry_mask default: True=Masked(Hidden), False=Visible.
# Kita ingin pixel di DALAM Jambi bernilai False (Visible/Valid).
geom_mask = geometry_mask(
    geometries=jambi_gdf.geometry,
    out_shape=(height_px, width_px),
    transform=out_transform,
    all_touched=True, # Opsional: Agar piksel yang tersentuh sedikit batas tetap masuk
    invert=False       # <--- UBAH KE TRUE AGAR AREA JAMBI TIDAK TERHAPUS
)

# D. Setup Transformer
transformer = Transformer.from_crs("EPSG:4326", TARGET_CRS, always_xy=True)

      Membuat Masking Geometri di Memori...


In [ ]:
# --- 3. EKSEKUSI LOOP ---

tif_files = sorted(glob.glob(os.path.join(INPUT_DIR, '*.ga_corrected.tif')))
print(f"[3/4] Ditemukan {len(tif_files)} file input.")
print("\n[4/4] Eksekusi Interpolasi & Saving...")

for tif_path in tqdm(tif_files, unit="file"):
    filename = os.path.basename(tif_path)

    # 1. Baca Data Sumber
    with rasterio.open(tif_path) as src:
        data_src = src.read(1)
        nodata_src = src.nodata if src.nodata is not None else -9999
        transform_src = src.transform

        # Optimasi: Hanya ambil koordinat dari piksel yang valid (bukan NoData)
        # Ini jauh lebih hemat memori daripada me-loop seluruh grid CHIRPS
        valid_rows, valid_cols = np.where((data_src != nodata_src) & (~np.isnan(data_src)))

        # Jika file kosong/rusak, skip
        if len(valid_rows) == 0:
            continue

        z_valid = data_src[valid_rows, valid_cols]

        # Dapatkan koordinat Lat/Lon untuk piksel valid saja
        lon_valid, lat_valid = rasterio.transform.xy(transform_src, valid_rows, valid_cols, offset='center')

    # 2. Konversi Koordinat Sumber ke UTM (Meter)
    x_utm_source, y_utm_source = transformer.transform(lon_valid, lat_valid)
    source_coords = np.c_[x_utm_source, y_utm_source]

    # 3. Interpolasi IDW (Menggunakan cKDTree)
    tree = cKDTree(source_coords)
    dist, idx = tree.query(target_coords, k=K_NEIGHBORS)

    # Hindari pembagian dengan nol
    dist = np.maximum(dist, 1e-10)

    weights = 1 / (dist ** IDW_POWER)
    numerator = np.sum(weights * z_valid[idx], axis=1)
    denominator = np.sum(weights, axis=1)

    z_interp_flat = numerator / denominator
    z_interp_2d = z_interp_flat.reshape((height_px, width_px))

    # 4. Terapkan Masking Geometri (Clipping)
    # Set nilai pixel di luar batas Jambi menjadi NoData
    z_interp_2d[geom_mask] = NODATA_VAL

    # 5. Simpan Langsung ke File Akhir
    final_name = filename.replace('.reg_corrected.tif', '.1000m_metric.tif')
    final_path = os.path.join(OUTPUT_DIR, final_name)

    out_meta = {
        'driver': 'GTiff',
        'height': height_px,
        'width': width_px,
        'count': 1,
        'dtype': 'float32',
        'crs': TARGET_CRS,
        'transform': out_transform,
        'nodata': NODATA_VAL,
        'compress': 'lzw' # Kompresi agar file tidak bengkak
    }

    with rasterio.open(final_path, 'w', **out_meta) as dst:
        dst.write(z_interp_2d.astype('float32'), 1)

print("\nSelesai! File output tersimpan di folder 'downscaled_1km_metric'.")

[3/4] Ditemukan 144 file input.

[4/4] Eksekusi Interpolasi & Saving...


100%|██████████| 144/144 [00:42<00:00,  3.40file/s]


Selesai! File output tersimpan di folder 'downscaled_1km_metric'.
